### Problem 1: Predicting Number of Attacks Next Month

In [1]:
import sklearn
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_theme (style="darkgrid")
print(sklearn.__version__)  # This will print the version of scikit-learn installed


1.6.1


In [4]:
df = pd.read_csv("linear_regression_dataset.csv")

prob1_dataset = df

In [5]:
prob1_dataset.head()

,time_start,launched,destroyed,model,launched_next_month
0,NaN,1.0,1.0,Молнія,1.0
1,NaN,1.0,1.0,Lancet,1.0
2,2025-04-10 16:04:00,1.0,0.0,Iskander-M,1.0
3,NaN,1.0,1.0,Orlan-10,2.0
4,NaN,2.0,2.0,Lancet,2.0
